In [1]:
%pip install imblearn.under_sampling

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement imblearn.under_sampling (from versions: none)
ERROR: No matching distribution found for imblearn.under_sampling


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from yahoo_fin.stock_info import get_data
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

%matplotlib inline

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\awhun\anaconda3\anaconda\Lib\site-packages\sklearn\utils\_param_validation.py)

In [ ]:
hist=get_data('DIS', index_as_date=False)

hist.head()

In [ ]:

# Remove the adjusted close column and rename our dataframe as "prices"

prices = hist.drop(['adjclose'], axis=1)
prices.head()

In [ ]:
# Add difference between closing price and opening price
# NOTE: O-to-C is the length of the candle's body

prices['O-to-C'] = prices['close'] - prices['open']

# Add 20-Day moving average for Open-to-Close column 

prices['OC-20D-Mean'] = prices['O-to-C'].rolling(20).mean()

# Calculate the % change of the current day's O-to-C relative to the moving average

prices['OC-%-from-20D-Mean'] = 100*(prices['O-to-C'] - prices['OC-20D-Mean'])/prices['OC-20D-Mean']

# Get the maximum OC compared to the recent 10 candles (including the current candle)

prices['MaxOC_Prev10'] = prices['O-to-C'].rolling(10).max()

# Add 20-Day moving average for volume 

prices['Volume-20D-Mean'] = prices['volume'].rolling(20).mean()

# Calculate the % change of the current volume relative to the moving average

prices['Volume-%-from-20D-Mean'] = 100*(prices['volume'] - prices['Volume-20D-Mean'])/prices['Volume-20D-Mean']

# Rearrange the columns for our dataframe

prices = prices[['ticker', 'date', 'open', 'high', 'low', 'close', 
                 'O-to-C', 'OC-20D-Mean', 'volume', 'Volume-20D-Mean', 
                 'MaxOC_Prev10', 'Volume-%-from-20D-Mean', 'OC-%-from-20D-Mean', 
                ]]

# Show the 10 most recent rows

prices.tail(10)

In [ ]:
# Remove rows with null values

prices = prices.dropna()
prices.info()

In [ ]:
#Isolating green candles
prices['O-to-C'] >= 0.0
#Separating bodies that are longest in 10 days
prices['O-to-C'] == prices['MaxOC_Prev10']
#Seeing if body is at 100% longer than average of previous 20 candles
prices['OC-%-from-20D-Mean'] >= 100.0
#Seeing if volume is at least 50% higher than average of previous 20 candles
prices['Volume-%-from-20D-Mean'] >= 50.0

In [ ]:
#Making a separate dataset for breakouts

condition = (prices['O-to-C'] >= 0.0) & (prices['O-to-C'] == prices['MaxOC_Prev10']) & (prices['OC-%-from-20D-Mean'] >= 100.0) & (prices['Volume-%-from-20D-Mean'] >= 50.0) 

breakouts = prices[condition]

breakouts

In [ ]:
#Creating a new column so we can assing binary values
prices['breakout_signal']=np.where(condition, 1,0)
prices.head()

In [ ]:
#Gets rid of null and infinite values
import pandas as pd
import numpy as np

def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)
    return df[indices_to_keep].astype(np.float64)

In [ ]:
#Making variables for X and y
X=filtered_df[['O-to-C','OC-20D-Mean','volume','Volume-20D-Mean','MaxOC_Prev10','Volume-%-from-20D-Mean','OC-%-from-20D-Mean']]
y=filtered_df['breakout_signal']
#Still trying to remove infinite values
X.replace([np.inf, -np.inf], np.nan, inplace=True)
X.fillna(999, inplace=True)

# Create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)



In [ ]:
#Defining the classifier
classifier = xgb.XGBClassifier()
#Changing label for y to fit data
le = LabelEncoder()
y_train = le.fit_transform(y_train)
#Fitting the data
classifier.fit(X_train,y_train)
#Making predictions
y_pred = classifier.predict(X_test)
print(classification_report(y_test,y_pred))

In [ ]:
xgb_acc = accuracy_score(y_test, y_pred)
print(xgb_acc)

In [ ]:
#Instantiate the Randomoversampler instance
random_oversampler=RandomOverSampler(random_state=1)

#Create X and Y oversampled
X_over,y_over=random_oversampler.fit_resample(X_train,y_train)

#Instantiate the RandomUnderSampler instance
rus_RandomUnderSampler(random_state=1)

#Create x and y undersamplerd
X_under,y_under=rus.fit_resample(X_train,y_train)